In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

from utils.roi_measures import mad, ssim
from utils.extraction import extract_patches
from utils.reconstruction import perform_voting

from wnet import generate_wnet_model

Using TensorFlow backend.


In [10]:
wparams = {}
wparams['input_channels'] = 2
wparams['output_channels'] = 1
wparams['latent_channels'] = 16
wparams['scale'] = [0.5, 0.5, 0.5]
wparams['use_combined_loss'] = True
wparams['patch_shape'] = (32, 32, 32)
# wparams['loss_weights'] = [1, 1.0/np.sqrt(32**4), 1]
wparams['loss_weights'] = [1, 1, 1.1]

segparams = {}
segparams['seg_model_filename'] = ''
segparams['seg_model_params_filename'] = ''
segparams['segmentation_classes'] = 4

model = generate_wnet_model(wparams, segparams)

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 1, 32, 32, 32 0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 1, 32, 32, 32 0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 1, 32, 32, 32 0           input_10[0][0]                   
__________________________________________________________________________________________________
multiply_3 (Multiply)           (None, 1, 32, 32, 32 0           input_9[0][0]                    
                                                                 lambda_2[0][0]                   
__________

In [12]:
import os
import nibabel as nib

from medical_data import diff_info

ref_pattern = 'OAS2_{0:04}_MR{1}_halfwayto_OAS2_{0:04}_MR{2}'
file_general_pattern = 'OAS2_{0:04}_MR{1}_to_OAS2_{0:04}_MR{2}'
dataset_location = '/mnt/harddisk/datasets/OASIS/REG/{}/{}.nii.gz'

step = (16, 16, 16)
threshold = np.int32(0.30 * np.prod(wparams['patch_shape'][:]))
seg_train = np.empty((0, 1, ) + wparams['patch_shape'])
ref_train = np.empty((0, 1, ) + wparams['patch_shape'])
out_train = np.empty((0, 1, ) + wparams['patch_shape'])
for i in range(100, 190) :
    print '{} :'.format(i),
    
    for j in range(1, 5) :
        diff = 0
        for k in range(j+1, 5) :
            ref_filename = dataset_location.format(
                file_general_pattern.format(i, j, k), ref_pattern.format(i, j, k))
            ref_prob_filename = dataset_location.format(
                file_general_pattern.format(i, j, k), ref_pattern.format(i, j, k) + '_brain_seg')
            mov_filename = dataset_location.format(
                file_general_pattern.format(i, j, k), ref_pattern.format(i, k, j))
            mov_prob_filename = dataset_location.format(
                file_general_pattern.format(i, j, k), ref_pattern.format(i, k, j) + '_seg')
            
            if not (os.path.exists(ref_filename) and os.path.exists(mov_filename)) :
                continue
            
            diff = diff + diff_info['OAS2_0{:03}_MR{}'.format(i, k)]
            
            if diff < 0.02:
                continue

            ######################################################################################
            volume_init = nib.load(ref_filename).get_data()
            
            mask_patches = extract_patches(volume_init != 0, wparams['patch_shape'], step)

            useful_patches = np.sum(mask_patches, axis=(1, 2, 3)) > threshold
            
            del mask_patches
            
            ref_patches = extract_patches(volume_init, wparams['patch_shape'], step)
            ref_patches = ref_patches[useful_patches].reshape((-1, 1, ) + wparams['patch_shape'])
            ######################################################################################
            ref_train = np.vstack((ref_patches, ref_train)).astype('float32')
            del ref_patches

            ######################################################################################
            volume_init = nib.load(mov_filename).get_data()
            
            mov_patches = extract_patches(volume_init, wparams['patch_shape'], step)
            mov_patches = mov_patches[useful_patches].reshape((-1, 1, ) + wparams['patch_shape'])
            ######################################################################################
            volume_init = nib.load(mov_prob_filename).get_data() == 1
            
            mov_prob_patches = extract_patches(volume_init, wparams['patch_shape'], step)
            mov_prob_patches = mov_prob_patches[useful_patches].reshape((-1, 1, ) + wparams['patch_shape'])
            ######################################################################################
            seg_train = np.vstack((mov_prob_patches, seg_train)).astype('float32') ##
            out_train = np.vstack((mov_patches, out_train)).astype('float32')
            del mov_patches, mov_prob_patches
            ######################################################################################
            print '{}->{} ({:.2f})'.format(j, k, diff),
    print

100 : 1->2 (0.02) 1->3 (0.02)
101 : 1->2 (0.02) 1->3 (0.03)
102 :
103 : 1->2 (0.03) 1->3 (0.04)
104 :
105 :
106 : 1->2 (0.04)
107 :
108 :
109 :
110 :
111 : 1->2 (0.03)
112 :
113 : 1->2 (0.02)
114 : 1->2 (0.02)
115 :
116 :
117 : 1->3 (0.02) 1->4 (0.03) 2->3 (0.02) 2->4 (0.03)
118 :
119 :
120 :
121 : 1->2 (0.02)
122 :
123 :
124 :
125 :
126 : 1->3 (0.02)
127 : 1->4 (0.03) 2->4 (0.02) 3->4 (0.02)
128 : 1->2 (0.02)
129 : 1->3 (0.02) 2->3 (0.02)
130 :
131 :
132 :
133 : 1->3 (0.04)
134 :
135 : 1->2 (0.02)
136 :
137 :
138 : 1->2 (0.02)
139 : 1->2 (0.03)
140 : 1->2 (0.02) 1->3 (0.02)
141 : 1->2 (0.02)
142 :
143 : 1->3 (0.02) 2->3 (0.02)
144 :
145 : 1->2 (0.03)
146 :
147 : 1->4 (0.03) 2->4 (0.03) 3->4 (0.02)
148 :
149 :
150 :
151 :
152 :
153 :
154 : 1->2 (0.04)
155 :
156 :
157 : 1->2 (0.07)
158 : 1->2 (0.02)
159 :
160 :
161 :
162 :
163 :
164 :
165 :
166 :
167 :
168 :
169 :
170 :
171 : 1->3 (0.02) 2->3 (0.02)
172 :
173 :
174 :
175 :
176 : 1->3 (0.03) 2->3 (0.02)
177 :
178 : 1->3 (0.02)
179 : 1->2

In [13]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

patience = 10

stopper = EarlyStopping(patience=patience)
checkpointer = ModelCheckpoint('models/ag_mseloss_o2o1.h5', save_best_only=True, save_weights_only=True)

N = len(ref_train)
model.fit(
    [ref_train, seg_train],
    [np.multiply(out_train, 1-seg_train), np.multiply(out_train, seg_train), out_train],
    validation_split=0.3, epochs=100, batch_size=32,
    callbacks=[checkpointer, stopper])

Train on 15439 samples, validate on 6618 samples
Epoch 1/100
15439/15439 [==============================] - 175s 11ms/step - loss: 123766.1595 - model_5_loss: 46531.6178 - model_6_loss: 11096.3193 - model_7_loss: 60125.6558 - val_loss: 54799.3429 - val_model_5_loss: 16479.8978 - val_model_6_loss: 7315.3727 - val_model_7_loss: 28185.5199
Epoch 2/100
15439/15439 [==============================] - 169s 11ms/step - loss: 66777.7236 - model_5_loss: 25367.2277 - model_6_loss: 6484.4508 - model_7_loss: 31750.9494 - val_loss: 45182.0661 - val_model_5_loss: 14524.5341 - val_model_6_loss: 6112.7068 - val_model_7_loss: 22313.4767
Epoch 3/100
15439/15439 [==============================] - 166s 11ms/step - loss: 57151.0072 - model_5_loss: 22019.9642 - model_6_loss: 6395.3411 - model_7_loss: 26123.3647 - val_loss: 43400.7574 - val_model_5_loss: 13566.2917 - val_model_6_loss: 6539.9532 - val_model_7_loss: 21176.8288
Epoch 4/100
15439/15439 [==============================] - 165s 11ms/step - loss: 466

15439/15439 [==============================] - 164s 11ms/step - loss: 16947.3167 - model_5_loss: 6265.4873 - model_6_loss: 2679.6173 - model_7_loss: 7274.7381 - val_loss: 28220.5281 - val_model_5_loss: 10595.6443 - val_model_6_loss: 3676.2495 - val_model_7_loss: 12680.5764
Epoch 30/100
15439/15439 [==============================] - 164s 11ms/step - loss: 20179.3663 - model_5_loss: 7378.7279 - model_6_loss: 3274.0839 - model_7_loss: 8660.5039 - val_loss: 29799.1213 - val_model_5_loss: 11137.6830 - val_model_6_loss: 3795.6394 - val_model_7_loss: 13514.3622
Epoch 31/100
15439/15439 [==============================] - 164s 11ms/step - loss: 18703.2135 - model_5_loss: 7064.1400 - model_6_loss: 2795.3002 - model_7_loss: 8039.7936 - val_loss: 30648.1259 - val_model_5_loss: 11400.2478 - val_model_6_loss: 3666.3763 - val_model_7_loss: 14165.0013
Epoch 32/100
15439/15439 [==============================] - 171s 11ms/step - loss: 17208.5135 - model_5_loss: 6535.4821 - model_6_loss: 2633.9971 - mode

In [14]:
model.load_weights('models/ag_mseloss_o2o1.h5')

In [15]:
from keras.models import Model
import os
import nibabel as nib
import SimpleITK as sitk

ref_pattern = 'OAS2_{0:04}_MR{1}_halfwayto_OAS2_{0:04}_MR{2}'
file_general_pattern = 'OAS2_{0:04}_MR{1}_to_OAS2_{0:04}_MR{2}'
dataset_location = '/mnt/harddisk/datasets/OASIS/REG/{}/{}.nii.gz'

mad_results = {}
ssim_results = {}
curr_patch_shape = (32, 32, 32)
step = (32, 32, 32)
for i in range(1, 100) :
    print '{} :'.format(i),
    for j in range(1, 5) :
        for k in range(j+1, 5) :
            ref_filename = dataset_location.format(
                file_general_pattern.format(i, j, k), ref_pattern.format(i, j, k))
            ref_prob_filename = dataset_location.format(
                file_general_pattern.format(i, j, k), ref_pattern.format(i, j, k) + '_brain_seg')
            mov_filename = dataset_location.format(
                file_general_pattern.format(i, j, k), ref_pattern.format(i, k, j))
            mov_prob_filename = dataset_location.format(
                file_general_pattern.format(i, j, k), ref_pattern.format(i, k, j) + '_seg')
            
            print ref_prob_filename
            
            if not (os.path.exists(ref_filename) and os.path.exists(mov_filename)) :
                continue
                
            ######################################################################################
            volume_init = nib.load(ref_filename).get_data()

            ref_patches = extract_patches(volume_init, curr_patch_shape, step)
            ref_patches = ref_patches.reshape((-1, 1, ) + curr_patch_shape)
            ######################################################################################
            volume_init = nib.load(mov_prob_filename).get_data() == 1
            
            mov_prob_patches = extract_patches(volume_init, curr_patch_shape, step)
            mov_prob_patches = mov_prob_patches.reshape((-1, 1, ) + curr_patch_shape)
            ######################################################################################
            print '{}->{}'.format(j, k),

            pred = model.predict(
                [ref_patches, mov_prob_patches], verbose=1, batch_size=64)[2]
            pred = pred.reshape((-1, ) + curr_patch_shape)

            volume = perform_voting(pred, curr_patch_shape, volume_init.shape, step)

            volume_data = nib.load(mov_filename)

            volume = np.multiply(volume_data.get_data() != 0, volume)
            
            save_filename = 'mse_results/{}_{}_to_{}.nii.gz'.format(i, j, k)
            save_filename_cor = 'mse_results/{}_{}_to_{}_cor.nii.gz'.format(i, j, k)

            nib.save(nib.Nifti1Image(volume, volume_data.affine), save_filename)

            res = sitk.ReadImage(save_filename)

            caster = sitk.CastImageFilter()
            caster.SetOutputPixelType(res.GetPixelID())

            orig = caster.Execute(sitk.ReadImage(mov_filename))
            seg = caster.Execute(sitk.ReadImage(mov_prob_filename))

            thresholder = sitk.BinaryThresholdImageFilter()
            enhanced_vol = caster.Execute(
                thresholder.Execute(seg, -1, -1, 1, 0))

            pairs = [(1, 1), (2, 3)]
            for (a, b) in pairs :                
                thresholder = sitk.BinaryThresholdImageFilter()
                mask = thresholder.Execute(seg, a, b, 1, 0)

                masker = sitk.MaskImageFilter()
                ref_masked = masker.Execute(orig, mask)
                moving_masked = masker.Execute(res, mask)

                matcher = sitk.HistogramMatchingImageFilter()
                matcher.SetNumberOfHistogramLevels(2048)
                matcher.SetNumberOfMatchPoints(15)
                matcher.SetThresholdAtMeanIntensity(True)
                partial_result = matcher.Execute(moving_masked, ref_masked)

                adder = sitk.AddImageFilter()
                enhanced_vol = adder.Execute(enhanced_vol, partial_result)

            masker = sitk.MaskImageFilter()
            mask = thresholder.Execute(seg, 0, 0, 0, 1)
            enhanced_vol = masker.Execute(enhanced_vol, mask)
            sitk.WriteImage(enhanced_vol, save_filename_cor)

            volume = nib.load(save_filename_cor).get_data()
            act_vol = np.float64(nib.load(mov_filename).get_data())
            mad_movgen = mad(volume, act_vol)
            ssim_movgen = ssim(volume, act_vol)

            mad_results['{} {}->{}'.format(i, j, k)] = mad_movgen
            ssim_results['{} {}->{}'.format(i, j, k)] = ssim_movgen
            print '{}-{}: {} - {}'.format(i, k, mad_movgen, ssim_movgen)

1 : /mnt/harddisk/datasets/OASIS/REG/OAS2_0001_MR1_to_OAS2_0001_MR2/OAS2_0001_MR1_halfwayto_OAS2_0001_MR2_brain_seg.nii.gz
320/320 [==============================] - 2s 7ms/step
 1-2: 0.0206712573647 - 0.968303241213
/mnt/harddisk/datasets/OASIS/REG/OAS2_0001_MR1_to_OAS2_0001_MR3/OAS2_0001_MR1_halfwayto_OAS2_0001_MR3_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0001_MR1_to_OAS2_0001_MR4/OAS2_0001_MR1_halfwayto_OAS2_0001_MR4_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0001_MR2_to_OAS2_0001_MR3/OAS2_0001_MR2_halfwayto_OAS2_0001_MR3_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0001_MR2_to_OAS2_0001_MR4/OAS2_0001_MR2_halfwayto_OAS2_0001_MR4_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0001_MR3_to_OAS2_0001_MR4/OAS2_0001_MR3_halfwayto_OAS2_0001_MR4_brain_seg.nii.gz
2 : /mnt/harddisk/datasets/OASIS/REG/OAS2_0002_MR1_to_OAS2_0002_MR2/OAS2_0002_MR1_halfwayto_OAS2_0002_MR2_brain_seg.nii.gz
320/320 [==============================] - 1s 3ms/step
 2-2: 0.03

320/320 [==============================] - 1s 3ms/step
 12-2: 0.0242604736912 - 0.968466510186
/mnt/harddisk/datasets/OASIS/REG/OAS2_0012_MR1_to_OAS2_0012_MR3/OAS2_0012_MR1_halfwayto_OAS2_0012_MR3_brain_seg.nii.gz
320/320 [==============================] - 1s 3ms/step
 12-3: 0.0258634676039 - 0.962588985635
/mnt/harddisk/datasets/OASIS/REG/OAS2_0012_MR1_to_OAS2_0012_MR4/OAS2_0012_MR1_halfwayto_OAS2_0012_MR4_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0012_MR2_to_OAS2_0012_MR3/OAS2_0012_MR2_halfwayto_OAS2_0012_MR3_brain_seg.nii.gz
320/320 [==============================] - 1s 3ms/step
 12-3: 0.0246864283748 - 0.967981876724
/mnt/harddisk/datasets/OASIS/REG/OAS2_0012_MR2_to_OAS2_0012_MR4/OAS2_0012_MR2_halfwayto_OAS2_0012_MR4_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0012_MR3_to_OAS2_0012_MR4/OAS2_0012_MR3_halfwayto_OAS2_0012_MR4_brain_seg.nii.gz
13 : /mnt/harddisk/datasets/OASIS/REG/OAS2_0013_MR1_to_OAS2_0013_MR2/OAS2_0013_MR1_halfwayto_OAS2_0013_MR2_brain_seg.nii

320/320 [==============================] - 1s 3ms/step
 22-2: 0.0205649303726 - 0.966784593595
/mnt/harddisk/datasets/OASIS/REG/OAS2_0022_MR1_to_OAS2_0022_MR3/OAS2_0022_MR1_halfwayto_OAS2_0022_MR3_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0022_MR1_to_OAS2_0022_MR4/OAS2_0022_MR1_halfwayto_OAS2_0022_MR4_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0022_MR2_to_OAS2_0022_MR3/OAS2_0022_MR2_halfwayto_OAS2_0022_MR3_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0022_MR2_to_OAS2_0022_MR4/OAS2_0022_MR2_halfwayto_OAS2_0022_MR4_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0022_MR3_to_OAS2_0022_MR4/OAS2_0022_MR3_halfwayto_OAS2_0022_MR4_brain_seg.nii.gz
23 : /mnt/harddisk/datasets/OASIS/REG/OAS2_0023_MR1_to_OAS2_0023_MR2/OAS2_0023_MR1_halfwayto_OAS2_0023_MR2_brain_seg.nii.gz
320/320 [==============================] - 1s 3ms/step
 23-2: 0.0208091503712 - 0.954250944077
/mnt/harddisk/datasets/OASIS/REG/OAS2_0023_MR1_to_OAS2_0023_MR3/OAS2_0023_MR1_halfwayto_OAS

320/320 [==============================] - 1s 3ms/step
 31-3: 0.0227756114109 - 0.967693304813
/mnt/harddisk/datasets/OASIS/REG/OAS2_0031_MR2_to_OAS2_0031_MR4/OAS2_0031_MR2_halfwayto_OAS2_0031_MR4_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0031_MR3_to_OAS2_0031_MR4/OAS2_0031_MR3_halfwayto_OAS2_0031_MR4_brain_seg.nii.gz
32 : /mnt/harddisk/datasets/OASIS/REG/OAS2_0032_MR1_to_OAS2_0032_MR2/OAS2_0032_MR1_halfwayto_OAS2_0032_MR2_brain_seg.nii.gz
320/320 [==============================] - 1s 3ms/step
 32-2: 0.018575268598 - 0.8369806013
/mnt/harddisk/datasets/OASIS/REG/OAS2_0032_MR1_to_OAS2_0032_MR3/OAS2_0032_MR1_halfwayto_OAS2_0032_MR3_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0032_MR1_to_OAS2_0032_MR4/OAS2_0032_MR1_halfwayto_OAS2_0032_MR4_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0032_MR2_to_OAS2_0032_MR3/OAS2_0032_MR2_halfwayto_OAS2_0032_MR3_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0032_MR2_to_OAS2_0032_MR4/OAS2_0032_MR2_halfwayto_OAS2_0

320/320 [==============================] - 1s 3ms/step
 40-3: 0.0264869749149 - 0.940914469181
/mnt/harddisk/datasets/OASIS/REG/OAS2_0040_MR2_to_OAS2_0040_MR4/OAS2_0040_MR2_halfwayto_OAS2_0040_MR4_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0040_MR3_to_OAS2_0040_MR4/OAS2_0040_MR3_halfwayto_OAS2_0040_MR4_brain_seg.nii.gz
41 : /mnt/harddisk/datasets/OASIS/REG/OAS2_0041_MR1_to_OAS2_0041_MR2/OAS2_0041_MR1_halfwayto_OAS2_0041_MR2_brain_seg.nii.gz
320/320 [==============================] - 1s 3ms/step
 41-2: 0.0214510615641 - 0.972303219192
/mnt/harddisk/datasets/OASIS/REG/OAS2_0041_MR1_to_OAS2_0041_MR3/OAS2_0041_MR1_halfwayto_OAS2_0041_MR3_brain_seg.nii.gz
320/320 [==============================] - 1s 3ms/step
 41-3: 0.0279985658775 - 0.964250224367
/mnt/harddisk/datasets/OASIS/REG/OAS2_0041_MR1_to_OAS2_0041_MR4/OAS2_0041_MR1_halfwayto_OAS2_0041_MR4_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0041_MR2_to_OAS2_0041_MR3/OAS2_0041_MR2_halfwayto_OAS2_0041_MR3_brain_seg.nii

...

320/320 [==============================] - 1s 3ms/step
 60-2: 0.0219650659104 - 0.961474989561
/mnt/harddisk/datasets/OASIS/REG/OAS2_0060_MR1_to_OAS2_0060_MR3/OAS2_0060_MR1_halfwayto_OAS2_0060_MR3_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0060_MR1_to_OAS2_0060_MR4/OAS2_0060_MR1_halfwayto_OAS2_0060_MR4_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0060_MR2_to_OAS2_0060_MR3/OAS2_0060_MR2_halfwayto_OAS2_0060_MR3_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0060_MR2_to_OAS2_0060_MR4/OAS2_0060_MR2_halfwayto_OAS2_0060_MR4_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0060_MR3_to_OAS2_0060_MR4/OAS2_0060_MR3_halfwayto_OAS2_0060_MR4_brain_seg.nii.gz
61 : /mnt/harddisk/datasets/OASIS/REG/OAS2_0061_MR1_to_OAS2_0061_MR2/OAS2_0061_MR1_halfwayto_OAS2_0061_MR2_brain_seg.nii.gz
320/320 [==============================] - 1s 3ms/step
 61-2: 0.022914164325 - 0.96337566313
/mnt/harddisk/datasets/OASIS/REG/OAS2_0061_MR1_to_OAS2_0061_MR3/OAS2_0061_MR1_halfwayto_OAS2_

320/320 [==============================] - 1s 3ms/step
 69-2: 0.0275334387742 - 0.970282623213
/mnt/harddisk/datasets/OASIS/REG/OAS2_0069_MR1_to_OAS2_0069_MR3/OAS2_0069_MR1_halfwayto_OAS2_0069_MR3_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0069_MR1_to_OAS2_0069_MR4/OAS2_0069_MR1_halfwayto_OAS2_0069_MR4_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0069_MR2_to_OAS2_0069_MR3/OAS2_0069_MR2_halfwayto_OAS2_0069_MR3_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0069_MR2_to_OAS2_0069_MR4/OAS2_0069_MR2_halfwayto_OAS2_0069_MR4_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0069_MR3_to_OAS2_0069_MR4/OAS2_0069_MR3_halfwayto_OAS2_0069_MR4_brain_seg.nii.gz
70 : /mnt/harddisk/datasets/OASIS/REG/OAS2_0070_MR1_to_OAS2_0070_MR2/OAS2_0070_MR1_halfwayto_OAS2_0070_MR2_brain_seg.nii.gz
320/320 [==============================] - 1s 3ms/step
 70-2: 0.0251011729209 - 0.943989250606
/mnt/harddisk/datasets/OASIS/REG/OAS2_0070_MR1_to_OAS2_0070_MR3/OAS2_0070_MR1_halfwayto_OAS

320/320 [==============================] - 1s 3ms/step
 78-2: 0.0199444311063 - 0.968452711248
/mnt/harddisk/datasets/OASIS/REG/OAS2_0078_MR1_to_OAS2_0078_MR3/OAS2_0078_MR1_halfwayto_OAS2_0078_MR3_brain_seg.nii.gz
320/320 [==============================] - 1s 3ms/step
 78-3: 0.0250487241079 - 0.951912101169
/mnt/harddisk/datasets/OASIS/REG/OAS2_0078_MR1_to_OAS2_0078_MR4/OAS2_0078_MR1_halfwayto_OAS2_0078_MR4_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0078_MR2_to_OAS2_0078_MR3/OAS2_0078_MR2_halfwayto_OAS2_0078_MR3_brain_seg.nii.gz
320/320 [==============================] - 1s 3ms/step
 78-3: 0.0241543414082 - 0.950356923813
/mnt/harddisk/datasets/OASIS/REG/OAS2_0078_MR2_to_OAS2_0078_MR4/OAS2_0078_MR2_halfwayto_OAS2_0078_MR4_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0078_MR3_to_OAS2_0078_MR4/OAS2_0078_MR3_halfwayto_OAS2_0078_MR4_brain_seg.nii.gz
79 : /mnt/harddisk/datasets/OASIS/REG/OAS2_0079_MR1_to_OAS2_0079_MR2/OAS2_0079_MR1_halfwayto_OAS2_0079_MR2_brain_seg.nii

320/320 [==============================] - 1s 3ms/step
 88-2: 0.0199790727101 - 0.960303148932
/mnt/harddisk/datasets/OASIS/REG/OAS2_0088_MR1_to_OAS2_0088_MR3/OAS2_0088_MR1_halfwayto_OAS2_0088_MR3_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0088_MR1_to_OAS2_0088_MR4/OAS2_0088_MR1_halfwayto_OAS2_0088_MR4_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0088_MR2_to_OAS2_0088_MR3/OAS2_0088_MR2_halfwayto_OAS2_0088_MR3_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0088_MR2_to_OAS2_0088_MR4/OAS2_0088_MR2_halfwayto_OAS2_0088_MR4_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0088_MR3_to_OAS2_0088_MR4/OAS2_0088_MR3_halfwayto_OAS2_0088_MR4_brain_seg.nii.gz
89 : /mnt/harddisk/datasets/OASIS/REG/OAS2_0089_MR1_to_OAS2_0089_MR2/OAS2_0089_MR1_halfwayto_OAS2_0089_MR2_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0089_MR1_to_OAS2_0089_MR3/OAS2_0089_MR1_halfwayto_OAS2_0089_MR3_brain_seg.nii.gz
320/320 [==============================] - 1s 3ms/step
 89-3: 0.024

320/320 [==============================] - 1s 3ms/step
 98-2: 0.0180928180519 - 0.963703952921
/mnt/harddisk/datasets/OASIS/REG/OAS2_0098_MR1_to_OAS2_0098_MR3/OAS2_0098_MR1_halfwayto_OAS2_0098_MR3_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0098_MR1_to_OAS2_0098_MR4/OAS2_0098_MR1_halfwayto_OAS2_0098_MR4_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0098_MR2_to_OAS2_0098_MR3/OAS2_0098_MR2_halfwayto_OAS2_0098_MR3_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0098_MR2_to_OAS2_0098_MR4/OAS2_0098_MR2_halfwayto_OAS2_0098_MR4_brain_seg.nii.gz
/mnt/harddisk/datasets/OASIS/REG/OAS2_0098_MR3_to_OAS2_0098_MR4/OAS2_0098_MR3_halfwayto_OAS2_0098_MR4_brain_seg.nii.gz
99 : /mnt/harddisk/datasets/OASIS/REG/OAS2_0099_MR1_to_OAS2_0099_MR2/OAS2_0099_MR1_halfwayto_OAS2_0099_MR2_brain_seg.nii.gz
320/320 [==============================] - 1s 3ms/step
 99-2: 0.0257481221338 - 0.95964043116
/mnt/harddisk/datasets/OASIS/REG/OAS2_0099_MR1_to_OAS2_0099_MR3/OAS2_0099_MR1_halfwayto_OAS2

In [16]:
from keras.models import Model
import os
import nibabel as nib
import SimpleITK as sitk

ref_pattern = 'OAS2_{0:04}_MR{1}_halfwayto_OAS2_{0:04}_MR{2}'
file_general_pattern = 'OAS2_{0:04}_MR{1}_to_OAS2_{0:04}_MR{2}'
dataset_location = '/mnt/harddisk/datasets/OASIS/REG/{}/{}.nii.gz'

tuples = [(1, 2, 1), (1, 2, 2), (1, 3, 3), (1, 4, 4)]
mad_results = {}
ssim_results = {}
curr_patch_shape = (32, 32, 32)
step = (32, 32, 32)
for i in range(1, 100) :
    print '{} :'.format(i),
    for (j, k, a) in tuples :
        if k != a :
            ref_filename = dataset_location.format(
                    file_general_pattern.format(i, j, k), ref_pattern.format(i, j, k))
            ref_prob_filename = dataset_location.format(
                file_general_pattern.format(i, j, k), ref_pattern.format(i, j, k) + '_brain_seg')
        else :
            ref_filename = dataset_location.format(
                file_general_pattern.format(i, j, k), ref_pattern.format(i, k, j))
            ref_prob_filename = dataset_location.format(
                file_general_pattern.format(i, j, k), ref_pattern.format(i, k, j) + '_seg')

        mov_filename = ref_filename
        mov_prob_filename = ref_prob_filename

        if not os.path.exists(ref_filename):
            continue

        ######################################################################################
        volume_init = nib.load(ref_filename).get_data()

        ref_patches = extract_patches(volume_init, curr_patch_shape, step)
        ref_patches = ref_patches.reshape((-1, 1, ) + curr_patch_shape)
        ######################################################################################
        volume_init = nib.load(ref_prob_filename).get_data() == 1

        ref_prob_patches = extract_patches(volume_init, curr_patch_shape, step)
        ref_prob_patches = ref_prob_patches.reshape((-1, 1, ) + curr_patch_shape)
        ######################################################################################
        ref_test = np.hstack((ref_patches, ref_prob_patches)).astype('float32') ##
        seg_test = ref_prob_patches.astype('float32') ##
        del ref_patches, ref_prob_patches
        ######################################################################################
        print '{}->{}'.format(a, a)

        pred = model.predict(
            [ref_test[:, 0:1], seg_test], verbose=1, batch_size=64)[2]
        pred = pred.reshape((-1, ) + curr_patch_shape)

        volume = perform_voting(pred, curr_patch_shape, volume_init.shape, step)

        volume = np.multiply(nib.load(mov_filename).get_data() != 0, volume)

        volume_data = nib.load(mov_filename)

        nib.save(nib.Nifti1Image(volume, volume_data.affine),
                 'mse_results/{}_{}_to_{}.nii.gz'.format(i, a, a))
        
        res = sitk.ReadImage('mse_results/{}_{}_to_{}.nii.gz'.format(i, a, a))

        caster = sitk.CastImageFilter()
        caster.SetOutputPixelType(res.GetPixelID())

        orig = caster.Execute(sitk.ReadImage(mov_filename))
        seg = caster.Execute(sitk.ReadImage(mov_prob_filename))

        thresholder = sitk.BinaryThresholdImageFilter()
        enhanced_vol = caster.Execute(
            thresholder.Execute(seg, -1, -1, 1, 0))

        pairs = [(1, 1), (2, 3)]
        for (l, h) in pairs :                
            thresholder = sitk.BinaryThresholdImageFilter()
            mask = thresholder.Execute(seg, l, h, 1, 0)

            masker = sitk.MaskImageFilter()
            ref_masked = masker.Execute(orig, mask)
            moving_masked = masker.Execute(res, mask)

            matcher = sitk.HistogramMatchingImageFilter()
            matcher.SetNumberOfHistogramLevels(2048)
            matcher.SetNumberOfMatchPoints(100)
            matcher.SetThresholdAtMeanIntensity(True)
            partial_result = matcher.Execute(moving_masked, ref_masked)

            adder = sitk.AddImageFilter()
            enhanced_vol = adder.Execute(enhanced_vol, partial_result)

        masker = sitk.MaskImageFilter()
        mask = thresholder.Execute(seg, 0, 0, 0, 1)
        enhanced_vol = masker.Execute(enhanced_vol, mask)
        sitk.WriteImage(
            enhanced_vol,
            'mse_results/{}_{}_to_{}_cor.nii.gz'.format(i, a, a))

        volume = nib.load('mse_results/{}_{}_to_{}_cor.nii.gz'.format(i, a, a)).get_data()
        act_vol = np.float64(nib.load(mov_filename).get_data())
        mad_movgen = mad(volume, act_vol)
        ssim_movgen = ssim(volume, act_vol)

        mad_results['{} {}->{}'.format(i, a, a)] = mad_movgen
        ssim_results['{} {}->{}'.format(i, a, a)] = ssim_movgen
        print '{}-{}: {} - {}'.format(i, a, mad_movgen, ssim_movgen)

1 : 1->1
320/320 [==============================] - 1s 3ms/step
1-1: 0.0165569439424 - 0.978559562859
2->2
320/320 [==============================] - 1s 3ms/step
1-2: 0.016757745787 - 0.979367974128
2 : 1->1
320/320 [==============================] - 1s 3ms/step
2-1: 0.0414045747404 - 0.942364056125
2->2
320/320 [==============================] - 1s 3ms/step
2-2: 0.0193558523657 - 0.978311541727
3->3
320/320 [==============================] - 1s 3ms/step
2-3: 0.0253466302044 - 0.978658061398
3 : 4 : 1->1
320/320 [==============================] - 1s 3ms/step
4-1: 0.0274777182251 - 0.93604395135
2->2
320/320 [==============================] - 1s 3ms/step
4-2: 0.018050219702 - 0.971553112772
5 : 1->1
320/320 [==============================] - 1s 3ms/step
5-1: 0.0163551672494 - 0.974359098959
2->2
320/320 [==============================] - 1s 3ms/step
5-2: 0.0151620192686 - 0.975066591719
3->3
320/320 [==============================] - 1s 3ms/step
5-3: 0.0168428031751 - 0.973244348517
6 :

2->2
320/320 [==============================] - 1s 3ms/step
43-2: 0.0184089330811 - 0.981583567348
44 : 1->1
320/320 [==============================] - 1s 3ms/step
44-1: 0.0196618386804 - 0.978428324354
2->2
320/320 [==============================] - 1s 3ms/step
44-2: 0.0179017187552 - 0.975094698378
3->3
320/320 [==============================] - 1s 3ms/step
44-3: 0.0180301760464 - 0.977113774601
45 : 1->1
320/320 [==============================] - 1s 3ms/step: 
45-1: 0.0168941688684 - 0.981202082366
2->2
320/320 [==============================] - 1s 3ms/step
45-2: 0.0174302859326 - 0.980299382952
46 : 1->1
320/320 [==============================] - 1s 3ms/step
46-1: 0.0163068333585 - 0.974088775574
2->2
320/320 [==============================] - 1s 3ms/step
46-2: 0.016750054255 - 0.977071786497
47 : 1->1
320/320 [==============================] - 1s 3ms/step
47-1: 0.0182703695399 - 0.980115495294
2->2
320/320 [==============================] - 1s 3ms/step
47-2: 0.0181982079006 - 0.97

79 : 1->1
320/320 [==============================] - 1s 3ms/step
79-1: 0.0315399033538 - 0.945433757368
2->2
320/320 [==============================] - 1s 3ms/step
79-2: 0.0139865072653 - 0.980534838979
3->3
320/320 [==============================] - 1s 3ms/step
79-3: 0.0193898382816 - 0.979941973793
80 : 1->1
320/320 [==============================] - 1s 3ms/step
80-1: 0.0279305551221 - 0.790067863148
2->2
320/320 [==============================] - 1s 3ms/step
80-2: 0.0201817416591 - 0.973742884995
3->3
320/320 [==============================] - 1s 3ms/step
80-3: 0.0176152841334 - 0.97299933676
81 : 1->1
320/320 [==============================] - 1s 3ms/step
81-1: 0.0179734952877 - 0.980984315225
2->2
320/320 [==============================] - 1s 3ms/step
81-2: 0.019301527018 - 0.978878124702
82 : 83 : 84 : 85 : 1->1
320/320 [==============================] - 1s 3ms/step: 
85-1: 0.0195518093195 - 0.976394613735
2->2
320/320 [==============================] - 1s 3ms/step
85-2: 0.018696